In [10]:
import tkinter as tk
from tkinter import messagebox
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import threading

# Função para obter os dados de clima (executada em segundo plano)
def obter_dados_clima():
    try:
        # Iniciar o driver do Chrome em modo headless
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Executar o Chrome em segundo plano
        driver = webdriver.Chrome(options=options)
        driver.get('https://tempoagora.uol.com.br/previsao-do-tempo/15-dias/cidade/558/saopaulo-sp')

        # Reduzir o tempo de espera para elementos específicos
        driver.implicitly_wait(0.5)  # Diminuir o tempo de espera

        # Captura o HTML gerado pela página
        html_content = driver.page_source
        
        # Cria o objeto BeautifulSoup para fazer o parsing do HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Localiza os dias e as temperaturas
        days_info = soup.find_all('section', class_='accordion-card -daily-infos _border-bl-light-1')
        flex_divs = soup.find_all('div', class_='_flex _align-center _align-start-sm _space-between _relative _margin-l-sm-20')
        
        weather_data = []
        data_hora_pesquisa = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Captura a data e hora da pesquisa
        
        for day, flex_div in zip(days_info, flex_divs):
            # Extrair a data
            date_div = day.find('div', class_='date-inside-circle') or day.find('div', class_='date-inside-circlewith-alert')
            date = date_div.text.strip() if date_div else 'Data não encontrada'
            
            # Extrair temperaturas
            temp_min = ''
            temp_max = ''
            temp_min_img = day.find('img', alt='Temperatura mínima')
            if temp_min_img:
                temp_min = temp_min_img.find_next('span').text.strip() if temp_min_img.find_next('span') else 'Temp min não encontrada'
            
            temp_max_img = day.find('img', alt='Temperatura máxima')
            if temp_max_img:
                temp_max = temp_max_img.find_next('span').text.strip() if temp_max_img.find_next('span') else 'Temp max não encontrada'
            
            # Extrair umidade
            content = flex_div.text.strip()
            humidity = [line for line in content.split('\n') if '%' in line]
            humidity = humidity[0] if humidity else 'Umidade não encontrada'
            
            # Adiciona o resultado no array
            weather_data.append({
                'Data': date,
                'Temperatura Mínima': temp_min,
                'Temperatura Máxima': temp_max,
                'Umidade': humidity,
                'Data e Hora da Pesquisa': data_hora_pesquisa  # Adiciona a data e hora da pesquisa aqui
            })
        
        # Criar DataFrame
        df = pd.DataFrame(weather_data)
        
        # Tentar abrir o arquivo existente e anexar novos dados
        try:
            existing_df = pd.read_excel('dados_clima.xlsx')
            df = pd.concat([existing_df, df], ignore_index=True)  # Concatenar com os dados existentes
        except FileNotFoundError:
            pass  # Caso o arquivo não exista, ele será criado
        
        # Exportar os dados para um arquivo Excel
        df.to_excel('dados_clima.xlsx', index=False)
        messagebox.showinfo("Sucesso", "Dados exportados com sucesso para 'dados_clima.xlsx'.")
        
        # Carregar os dados na Listbox (usando thread)
        carregar_dados_na_listbox(weather_data, data_hora_pesquisa)
    
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro: {e}")
    
    finally:
        driver.quit()

# Função para carregar os dados na Listbox de maneira otimizada
def carregar_dados_na_listbox(weather_data, data_hora_pesquisa):
    def atualizar_listbox():
        # Inserir uma linha de separação com a data e hora da nova pesquisa
        listbox.insert(tk.END, f"------------------------ NOVA PESQUISA ({data_hora_pesquisa}) ------------------------")
        
        # Adicionar o cabeçalho na Listbox se estiver vazio
        if listbox.size() == 0:
            listbox.insert(tk.END, f"{'Data':<10} {'Temp Mínima':<15} {'Temp Máxima':<15} {'Umidade':<15}")
        
        # Adicionar os dados na Listbox sem apagar os anteriores
        for item in weather_data:
            # Alinhando os dados com f-strings
            listbox.insert(tk.END, f"{item['Data']:<10} {item['Temperatura Mínima']:<15} {item['Temperatura Máxima']:<15} {item['Umidade']:<15}")

    # Chamar a atualização no thread principal
    root.after(0, atualizar_listbox)

# Função que roda a obtenção de dados de clima em uma thread separada
def iniciar_pesquisa():
    threading.Thread(target=obter_dados_clima).start()

# Criar a interface gráfica
root = tk.Tk()
root.title("Consulta de Previsão do Tempo")


# Adicionar um botão para iniciar a pesquisa
botao_pesquisar = tk.Button(root, text="Obter Previsão", command=iniciar_pesquisa)
botao_pesquisar.pack(pady=20)

# Adicionar uma Listbox para exibir os dados do clima
listbox = tk.Listbox(root, width=100, height=20)
listbox.pack(pady=10)

# Rodar a interface gráfica
root.mainloop()


AttributeError: module 'tkinter' has no attribute 'ProgressBar'